In [7]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import re
import string
from typing import Counter
from NBClassifier import NBClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

Lấy dữ liệu từ file csv

In [8]:
df = pd.read_csv('SMSSpamCollection',sep='\t',header=None)
df.columns = ['Label', 'Mail']
df['Label'] = df['Label'].replace(['spam'],1)
df['Label'] = df['Label'].replace(['ham'],0)

# df = pd.read_csv('Emails.csv')
# df.columns = ['Label', 'Mail']
print(df.shape)
df.head()


(5572, 2)


,Label,Mail
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


Loại bỏ các mail trùng lặp

In [9]:
df.drop_duplicates(inplace = True)
print(df.shape)

(5169, 2)


Tách dataframe thành 2 tập train và test

In [10]:
randomized = df.sample(frac=1)
train_size = round(len(randomized) * 0.8)
train = randomized[:train_size].reset_index(drop=True)
test = randomized[train_size:].reset_index(drop=True)
print(train.shape)
print(test.shape)

(4135, 2)
(1034, 2)


Hàm tiền xử lý dữ liệu bao gồm: xóa các đường dẫn, xóa dấu câu, xóa các stopword

In [11]:
def process_text(text):
    text = re.sub(r'http\S+', '', text)
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    
    text = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    
    return text

Xử lí tập train, test thành vector

In [12]:
bag_word = []
testmails= test['Mail']
test_labels = test['Label']
testmails2 = []
trainmails = train['Mail']
train_labels = train['Label']
trainmails2= []

testmails = testmails.apply(process_text)
trainmails = trainmails.apply(process_text)




for doc in trainmails:
    words = doc
    bag_word += words

bag_word = set(bag_word)
bag_word = list(bag_word)

def tovector(mail):   
    splitmail = mail
    mvec = np.zeros(len(bag_word))
    count = Counter(splitmail)
    for i, word in enumerate(bag_word):
                mvec[i]+=count[word]
    
    return mvec

for i,mail in enumerate(testmails):

    testmails2.append(tovector(mail))

for i,mail in enumerate(trainmails):

    trainmails2.append(tovector(mail))

d = {'Mail': trainmails2, 'Label': train_labels}
Trainingdata = pd.DataFrame(data=d)
d = {'Mail': testmails2, 'Label': test_labels}
Testdata = pd.DataFrame(data=d)



Lọc thư rác bằng mô hình Naive Bayes (không dùng thư viện)

In [13]:
model= NBClassifier()
model.fit(Trainingdata['Mail'],Trainingdata['Label'])
result = model.predict(Testdata)

model.score(Testdata)

0.9777562862669246

In [14]:
trainingmails = list(Trainingdata['Mail'])
traininglabels= list(Trainingdata['Label'])
testmails=list(Testdata['Mail'])
testlabels= list(Testdata['Label'])

Lọc thư rác bằng mô hình Naive Bayes (có dùng thư viện)

In [15]:
model1 = MultinomialNB()
model1.fit(trainingmails,traininglabels)
result1 = model1.predict(testmails)
model1.score(testmails,testlabels)

0.9777562862669246

Lọc thư rác bằng mô hình Logistic Regression

In [16]:
model2 = LogisticRegression()
model2.fit(trainingmails,traininglabels)
result2 = model2.predict(testmails)
model2.score(testmails,testlabels)

0.9748549323017408

Lọc thư rác bằng mô hình SVC

In [17]:
model3 = SVC(kernel = 'linear')
model3.fit(trainingmails,traininglabels)
result3 = model3.predict(testmails)
model3.score(testmails,testlabels)

0.9806576402321083

Lọc thư rác bằng mô hình Random Forest

In [18]:
model4=RandomForestClassifier(n_estimators=100)
model4.fit(trainingmails,traininglabels)
result4 = model4.predict(testmails)
model4.score(testmails,testlabels)

0.9709864603481625